In [1]:
#.Make plots from Gkyl data.

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
from pylab import *
import postgkyl as pg
import numpy as np
import adios as ad
import sys

from shutil import copyfile
import pgkylUtil as pgu
import os
import subprocess
import shlex
import re

In [2]:
!ls ../v7/ | head -15

Biskamp_newNoise_FixedJ_Tratio100_elc_0.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_1.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_3.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_4.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_5.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_6.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_7.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_8.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_9.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_0.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_1.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_2.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_3.bp
Biskamp_newNoise_FixedJ_Tratio100_elc_M0_4.bp


In [3]:
#sys.path.insert(0, '/Users/rlw/Dropbox/ForBB/LucioFiles/pgkyl_script/')  

data_dir = '../v7'
input_filename   = 'Biskamp_newNoise_FixedJ_Tratio100.lua'   #.Root name of files to process. #todo(Ryan): fix hardcode

outDir     = '../gkyl_plots_v8_refactor/'               #.Output directory where figures are saved.
fourier_transform = True
auto_loading      = False
#creating the directory for plots if it does not exist yet
if not os.path.exists(outDir):
    os.makedirs(outDir)

In [4]:
# def main_plotting(fileroot, max_output):
#     print("Hello")

Making sure my regular expression works

In [5]:
file_prefix = "Biskamp_newNoise_FixedJ_Tratio100"
el = re.compile(rf'{file_prefix}_elc_\d+.bp')
#el = re.compile(r'Biskamp_newNoise_FixedJ_Tratio100_elc_\d+.bp')
el.search("Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp")

<re.Match object; span=(0, 42), match='Biskamp_newNoise_FixedJ_Tratio100_elc_2.bp'>

In [6]:
def get_max_filenumber(data_dir: str, input_filename: str) -> int:
    """Return the filenumber of the largest numbered output file in <data_dir> generated by <input_filename>
    
    data_dir: path to directory containing .bp files
    input_filename: e.g. 'Biskamp_2x2v.lua'
    """

    file_prefix = input_filename.split('.lua', maxsplit=1)[0]
    
    elc_pattern = re.compile(rf'{file_prefix}_elc_\d+.bp')
    elc_output_filenames = [filename for filename in os.listdir(data_dir) if elc_pattern.search(filename)]
    
    filenumbers = []
    for filename in elc_output_filenames:
            filenumber = re.findall(r'\d+.bp', filename)[0].split('.bp',1)[0]
            filenumbers.append(filenumber)
    
    max_filenumber = int(max(filenumbers))
    return max_filenumber

In [7]:
max_filenumber = get_max_filenumber(data_dir, input_filename)
max_filenumber

9

In [8]:
fileroot = input_filename.split('.lua', maxsplit=1)[0]
fileroot

'Biskamp_newNoise_FixedJ_Tratio100'

# The full `main_plotting` function

In [12]:
basisType

'ms'

## `get_grid_data`

In [13]:
def get_grid_data(fileName, nFr):
    #.Extract grid details from one of the data files for each species
    fName_elc = fileName+'_elc_'+str(nFr)+'.bp'
    fName_ion = fileName+'_ion_'+str(nFr)+'.bp'

    # getGrid data
    x_elc, _, nx, lx, _ = pgu.getGrid(fName_elc,polyOrder,basisType,location='center')
    x_ion, _, _, _, _ = pgu.getGrid(fName_ion,polyOrder,basisType,location='center')
            #x_e, gridDim, nx, lx, dx = pgu.getGrid(fName,polyOrder,basisType,location='center')
            #    x_e = np.array(x_e)  #RLW: Not needed

    #Store needed data from getGrid
    nz = nx[0]
    ny = nx[1]

    lz = lx[0]  #get box length along z, needed for Fourier transform
    ly = lx[1]  #get box length along y

    points_z = np.array(x_elc[0])
    points_y = np.array(x_elc[1])
    velocitiesz_elc = np.array(x_elc[2])  #attempt!!
    velocitiesy_elc = np.array(x_elc[3])  #attempt!!
    velocitiesz_ion = np.array(x_ion[2])  #attempt!!
    velocitiesy_ion = np.array(x_ion[3])  #attempt!!


    # Setting up the grids in V space for plotting
    #     electrons
    Vz_elc, Vy_elc = np.meshgrid(velocitiesz_elc,velocitiesy_elc,indexing='ij')
    #     ions
    Vz_ion, Vy_ion = np.meshgrid(velocitiesz_ion,velocitiesy_ion,indexing='ij')

    # Setting up the grids in X space for plotting
    grid_z, grid_y = np.meshgrid(points_z, points_y,indexing='ij')
    
    kz_plot_2d_sp   = 2.0*3.14159*vTe0*np.linspace(-int(nz/2), int(nz/2-1), nz)/lz
    ky_plot_2d_sp   = 2.0*3.14159*vTe0*np.linspace(-int(ny/2), int(ny/2-1), ny)/ly
    K_z, K_y = np.meshgrid(kz_plot_2d_sp, ky_plot_2d_sp, indexing = 'xy')
    
    return (Vz_elc, Vy_elc, Vz_ion, Vy_ion, grid_z, grid_y, nz, ny, K_z, K_y)

In [14]:
#Vz_elc, Vy_elc, Vz_ion, Vy_ion, grid_z, grid_y = get_grid_data(iFrame)
get_grid_data(fileName, 0)

/Users/rlw/anaconda3/envs/pgkyl-dev/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


(array([[-0.1191666666666667, -0.1191666666666667, -0.1191666666666667,
         ..., -0.1191666666666667, -0.1191666666666667,
         -0.1191666666666667],
        [-0.1175            , -0.1175            , -0.1175            ,
         ..., -0.1175            , -0.1175            ,
         -0.1175            ],
        [-0.1158333333333333, -0.1158333333333333, -0.1158333333333333,
         ..., -0.1158333333333333, -0.1158333333333333,
         -0.1158333333333333],
        ...,
        [ 0.1158333333333333,  0.1158333333333333,  0.1158333333333333,
         ...,  0.1158333333333333,  0.1158333333333333,
          0.1158333333333333],
        [ 0.1175            ,  0.1175            ,  0.1175            ,
         ...,  0.1175            ,  0.1175            ,
          0.1175            ],
        [ 0.1191666666666667,  0.1191666666666667,  0.1191666666666667,
         ...,  0.1191666666666667,  0.1191666666666667,
          0.1191666666666667]]),
 array([[-0.1191666666666667, -

## `get_species_data`

In [15]:
def get_species_data(fileName, nFr, species_code, mass=1.0, poly_order=2, basis_type='ms'):
    """Return interpolated distribution function data (and moments) for chosen species at chosen frame number
    """

    #     distribution function
    filename_f = fileName+'_'+ species_code + '_'+str(nFr)+'.bp'
    
    #     moments
    filename_M0 = fileName + '_' +  species_code + '_M0_'+str(nFr)+'.bp'
    filename_M1 = fileName+ '_' +  species_code + '_M1i_'+str(nFr)+'.bp'        
    filename_vT = fileName+ '_' +  species_code + '_vthSq_'+str(nFr)+'.bp'
    filename_u = fileName+ '_' +  species_code + '_u_'+str(nFr)+'.bp'

    
    f = np.squeeze(pgu.getInterpData(filename_f, poly_order, basis_type))
    n = np.squeeze(pgu.getInterpData(filename_M0, poly_order, basis_type, comp=0))
    nu_z = np.squeeze(pgu.getInterpData(filename_M1, poly_order, basis_type, comp=0))
    nu_y = np.squeeze(pgu.getInterpData(filename_M1, poly_order,basis_type, comp=1))
    u_z = np.squeeze(pgu.getInterpData(filename_u, poly_order,basis_type, comp=0))
    u_y = np.squeeze(pgu.getInterpData(filename_u, poly_order,basis_type, comp=1))
    vTsq = np.squeeze(pgu.getInterpData(filename_vT, poly_order, basis_type))
    
    vT = np.sqrt(vTsq)
    T = mass * vTsq/((vTe0)**2)  #This is an array of temperature values: one for each spatial location
    T_boxavg = np.average(T)
    return (f, n, nu_z, nu_y, u_z, u_y, vTsq, vT, T, T_boxavg)

## `get_field_data`

In [16]:
def get_field_data(fileName, nFr, poly_order=2, basis_type='ms'):
    """Return interpolated field data for the given frame number"""
    fName_field = fileName+'_'+ 'field' + '_'+str(nFr)+'.bp'
    eField_z = np.squeeze(pgu.getInterpData(fName_field,polyOrder,basisType,comp=0)) # JJ: can this be turned into 1 call without comp specified?
    eField_y = np.squeeze(pgu.getInterpData(fName_field,polyOrder,basisType,comp=1))
    
    return (eField_z, eField_y)

## `get_time`

In [17]:
def get_time(fileName, nFr):
                #.Extract the time from file.
    with ad.file(fileName+'_elc_'+str(nFr)+'.bp') as hF:
        time = hF['time'].read()
    return time

In [34]:
test_luafile = "Biskamp_newNoise_FixedJ_Tratio100.lua"

In [38]:
filepath = os.path.join(data_dir, test_luafile)
with open(filepath) as fh:
    dat = fh.read()
    
dat.split('\n')

['-- Gkyl ------------------------------------------------------------------------',
 '--Notes',
 '-- 0. The following modifications have been made ',
 '--      0.1 Get diagnostic temperature output for electrons ',
 '--      0.2 Unfortunately, this input file was a modification of an input file that had a "bad" noise function.  The waves have the form sin(pi x) rather than sin(2 pi x), so there is a wave whose period is too large, probably causing boundary issues.',
 '-- 1. Biskamp had a total run time of t = 2000 omega_pe^-1.  His plots show that the fluctuation energy increases until t = 800-1000, and then there is an ion heating phase, which is also a runaway production phase.',
 '--      1.1. Biskamp used a mass ratio of 400 for the plots shown.  The meaningful time is omega_pi, so t = 2000 (Sqrt[m_e/m_i]Biskamp) omega_pe^-1 = 2000*Sqrt(25/400) omega_pe^-1 = 2000/4 omega_pe^-1 = 500 omega_pe^-1. ',
 '--      1.2. We can run the sim for 1/2 the time, make plots, then run for the re

In [ ]:
basis       = "serendipity",    -- One of "serendipity" or "maximal-order".',
polyOrder   = pOrder,

In [ ]:
def get_luafile_data(luafile):
    filepath = os.path.join(data_dir, test_luafile)
    with open(filepath) as fh:
        fh.read()

In [33]:
np.load(outDir+fileName+rf'_timesFile'+'.npy')

array([0.                , 0.0646875         , 0.1265624999999999,
       0.1912499999999998, 0.2531249999999998, 0.3149999999999997,
       0.3796874999999996, 0.4415624999999995, 0.5034374999999994,
       0.5681249999999993])

In [32]:
!ls ../gkyl_plots_v8_refactor

Biskamp_newNoise_FixedJ_Tratio100_Telc_boxavgFile.npy
Biskamp_newNoise_FixedJ_Tratio100_Tion_boxavgFile.npy
Biskamp_newNoise_FixedJ_Tratio100_distributions_0000.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0001.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0002.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0003.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0004.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0005.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0006.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0007.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0008.png
Biskamp_newNoise_FixedJ_Tratio100_distributions_0009.png
Biskamp_newNoise_FixedJ_Tratio100_eSquaredFile.npy
Biskamp_newNoise_FixedJ_Tratio100_eZboxAvgFile.npy
Biskamp_newNoise_FixedJ_Tratio100_energy_current.png
Biskamp_newNoise_FixedJ_Tratio100_fft_0000.png
Biskamp_newNoise_FixedJ_Tratio100_fft_0001.png
Biskamp_newNoise_FixedJ_Tratio100_fft_0002.png
Biskamp_newNoise_FixedJ_Tr

In [9]:
os.chdir(data_dir)

In [10]:
fileName = fileroot
max_output = max_filenumber

handjam the data from the input file

In [11]:
# numerical
polyOrder  = 2                   #todo(Ryan): fix hardcode
basisType  = 'ms'                #todo(Ryan): fix hardcode

# physical
m_ion = 25                       #todo(Ryan): fix hardcode
vTe0 = 0.02                      #todo(Ryan): fix hardcode
alpha = 0.00                     #todo(Ryan): fix hardcode
cSound0 = vTe0/np.sqrt(m_ion) 

#.Component of the quantity we wish to extract from data file.
#.For field files this specifies the field component (e.g. Ex,
#.Ey, Ez, Bx, By, or Bz) while for Mi1 it specifies the vector
#.component of the momentum density.
compZero = 0

In [18]:
test_nFr = 2
test_fileName = fileName
test_time = get_time(fileName, test_nFr)

<ipython-input-17-bb831677cd37>:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  time = hF['time'].read()


In [19]:
# eField_z, eField_y = get_field_data(2)

In [29]:
iFrame     = 0     #.Initial frame (0 is the t=0 frame).
fFrame     = max_output    #.Final frame.
nFrames = fFrame-iFrame+1    #.Number of frames plotted.


Vz_elc, Vy_elc, Vz_ion, Vy_ion, grid_z, grid_y, nz, ny, K_z, K_y = get_grid_data(fileName, iFrame)
#initialize all arrays containing time-dependent quantities...that are going to be filled in during loop over frames

    # np.save(outDir+fileName+rf'_timesFile',times[0:fFrame-iFrame+1])
    # np.save(outDir+fileName+rf'_eZboxAvgFile',eField_boxavg_z[0:fFrame-iFrame+1])
    # np.save(outDir+fileName+rf'_jZboxAvgFile',J_boxavg_z[0:fFrame-iFrame+1])
    # np.save(outDir+fileName+rf'_eSquaredFile',eField_fluct_squared[0:fFrame-iFrame+1])
    # np.save(outDir+fileName+rf'_Tion_boxavgFile',Tion_boxavg[0:fFrame-iFrame+1])
    # np.save(outDir+fileName+rf'_Telc_boxavgFile',Telc_boxavg[0:fFrame-iFrame+1])

times = np.zeros(nFrames)

eField_boxavg_z = np.zeros(nFrames)
Rei = np.zeros(nFrames)
J_boxavg_z = np.zeros(nFrames)
eField_fluct_squared   = np.zeros(nFrames)
eField_fluct_squared_byT = np.zeros(nFrames)
E_over_J_rolling = np.zeros(nFrames)
R_over_J_rolling = np.zeros(nFrames)
energy_e_tot = np.zeros(nFrames)
energy_b_tot = np.zeros(nFrames)
E_boxavg_over_J_boxavg = np.zeros(nFrames)
Tion_boxavg = np.zeros(nFrames)
Telc_boxavg = np.zeros(nFrames)
nu_Sagdeev = np.zeros(nFrames)


for nFr in range(iFrame,fFrame+1):
    
    # Get filenames for n
    # electron filenames
    #     distribution
    elcd, elcM0, elcM1_z, elcM1_y, elc_u_z, elc_u_y, elc_vTsq, elc_vT, elcT, elcT_boxavg = get_species_data(fileName, nFr, 'elc', poly_order=2, basis_type='ms')
    iond, ionM0, ionM1_z, ionM1_y, ion_u_z, ion_u_y, ion_vTsq, ion_vT, ionT, ionT_boxavg = get_species_data(fileName, nFr, 'ion', mass=m_ion, poly_order=2, basis_type='ms')
    eField_z, eField_y = get_field_data(fileName, nFr, poly_order=2, basis_type='ms') 
#     # ion filenames
#     #     distribution
#     fName_ion = fileName+'_ion_'+str(nFr)+'.bp'
#     #     Moments
#     fNameM0_ion = fileName+'_ion_M0_'+str(nFr)+'.bp'
#     fNameM1_ion = fileName+'_ion_M1i_'+str(nFr)+'.bp'        
#     fName_vT_ion = fileName+'_ion_vthSq_'+str(nFr)+'.bp'
#     fName_u_ion = fileName+'_ion_u_'+str(nFr)+'.bp'

#     # electromagnetic field  filename
    


# #        elcM2 = np.squeeze(pgu.getRawData(fNameM2_elc))


#     iond = np.squeeze(pgu.getInterpData(fName_ion,polyOrder,basisType))
#     ionM0 = np.squeeze(pgu.getInterpData(fNameM0_ion,polyOrder,basisType,comp=0))	# JJ: are we sure that we have to specify the keyword comp
#     ionM1_z = np.squeeze(pgu.getInterpData(fNameM1_ion,polyOrder,basisType,comp=0))
#     ionM1_y = np.squeeze(pgu.getInterpData(fNameM1_ion,polyOrder,basisType,comp=1))
#     ion_u_z = np.squeeze(pgu.getInterpData(fName_u_ion,polyOrder,basisType,comp=0))
#     ion_u_y = np.squeeze(pgu.getInterpData(fName_u_ion,polyOrder,basisType,comp=1))
#     # ionM1_raw = np.squeeze(pgu.getRawData(fNameM1_ion)) RLW: commented this out because getRawData has problems in pgkylUtil.
#     ion_vTsq = np.squeeze(pgu.getInterpData(fName_vT_ion, polyOrder, basisType))
#     ionT = m_ion * ion_vTsq/((vTe0)**2)  #This is an array of temperature values: one for each spatial location
#     ionT_boxavg = np.average(ionT)


    
#     eField_z = np.squeeze(pgu.getInterpData(fName_field,polyOrder,basisType,comp=0)) # JJ: can this be turned into 1 call without comp specified?
#     eField_y = np.squeeze(pgu.getInterpData(fName_field,polyOrder,basisType,comp=1))

    
    
    elcd_box_avg = np.average(elcd, axis = (0,1)) #average over both spatial dimensions
    iond_box_avg = np.average(iond, axis = (0,1))

    # get z coordinate of center of box (it is 48)
    z0_elc = int(elcd.shape[0]/2)
    y0_elc = int(elcd.shape[1]/2)

    z0_ion = int(iond.shape[0]/2)
    y0_ion = int(iond.shape[1]/2)
    
    # get distribution function data for center of box
    elcd_cut = elcd[z0_elc,y0_elc,:,:]
    iond_cut = iond[z0_ion,y0_ion,:,:]

    # eName = fileName+'_field_'+str(nFr)+'.bp'

    #temperature plotting
    #Ti_fname = fileName + '_ion_' + 'intM2Thermal_' + str(nFr) + '.bp'
    #ionM2 = np.squeeze(pgu.getRawData(Ti_fname))
    #        ionM2 = np.squeeze(pgu.getRawData(Ti_fname,polyOrder,basisType,comp=compZero))
    #Jincreasing_simplest_randomkicksfunJ_Tratio100_standard2D_RLW_LMM_edit3_diag_test_ion_intM2Thermal

    #loading electric field

    #split into spatial average and fluctuating parts
    boxavg_eField_z = np.average(eField_z)
    boxavg_eField_y = np.average(eField_y)
    eField_fluct_z = eField_z - boxavg_eField_z
    eField_fluct_y = eField_y - boxavg_eField_y
    #.Compute the kz=0 component of the electric field
    eField_boxavg_z[nFr-iFrame] = boxavg_eField_z
    
    boxavg_uElc_z = np.average(elc_u_z)
    boxavg_uElc_y = np.average(elc_u_y)

    
    #compute energy in electric and magnetic fields field
    Tion_boxavg[nFr-iFrame] = ionT_boxavg
    Telc_boxavg[nFr-iFrame] = elcT_boxavg
    #print("ratio of averages")
    #print((np.sum(eField_fluct_z**2 + eField_fluct_y**2) /(nz*ny) )/np.average(eField_fluct_z**2 + eField_fluct_y**2))
    #eField_fluct_squared[nFr-iFrame] = (np.sum(eField_fluct_z**2 + eField_fluct_y**2) /(nz*ny) ) / vTe0**2
    eField_fluct_squared[nFr-iFrame] = np.average(eField_fluct_z**2 + eField_fluct_y**2)/vTe0**2
    eField_fluct_squared_byT[nFr-iFrame] = eField_fluct_squared[nFr-iFrame]/(elcT_boxavg)**2

    


    #.Compute current density.
    Jz     = ionM1_z - elcM1_z
    Jy     = ionM1_y - elcM1_y
    n_ion = ionM0
    J_fluct_z = Jz - np.average(Jz)
    J_fluct_y = Jy - np.average(Jy)

    #updating J_boxavg_z
    J_boxavg_z[nFr-iFrame] = np.average(Jz)

    #             #.Extract the time from file.
    #     hF         = ad.file(fileName+'_elc_'+str(nFr)+'.bp')
    #     time = hF['time'].read()
    #     hF.close()
    time = get_time(fileName, nFr)
    times[nFr-iFrame] = time
    
    time = float('%.3g' % times[nFr-iFrame])  #fix me please

    fignum = str(nFr).zfill(4)


    
    #Distribution function plots [Dplots]
    fig, axs = plt.subplots(2,3,figsize=(40, 20), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace =.1)
    axs = axs.ravel()

    pos0 = axs[0].pcolormesh(Vz_elc/cSound0, Vy_elc/cSound0, elcd_cut)
    axs[0].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[0].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[0].set_title(r'$F_e(z_0,y_0,v_z,v_y),$' + rf'$J_z$ = {np.average(Jz)}/{vTe0}'+ r' [$c_{s0}$]', fontsize=26)
    axs[0].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos0, ax=axs[0])
    cbar.ax.tick_params(labelsize=22)

    pos1 = axs[1].pcolormesh(Vz_ion/cSound0, Vy_ion/cSound0, iond_cut)
    axs[1].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[1].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[1].set_title(r'$F_i(z_0,y_0,v_z,v_y),$' + rf't = {time}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[1].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos1, ax=axs[1])
    cbar.ax.tick_params(labelsize=22)

    pos2 = axs[2].pcolormesh(grid_y, grid_z, Jz)
    axs[2].set_xlabel(r'$y \ [d_e]$', fontsize=30)
    axs[2].set_ylabel(r'$z \ [d_e]$', fontsize=30, labelpad=-1)
    axs[2].set_title(r'$J_z(z,y)$', fontsize=30)
    axs[2].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos2, ax=axs[2])
    cbar.ax.tick_params(labelsize=22)

    pos3 = axs[3].pcolormesh(Vz_elc/cSound0, Vy_elc/cSound0, elcd_box_avg)
    axs[3].scatter(boxavg_uElc_z, boxavg_uElc_y, s = 60)
    axs[3].scatter(np.squeeze(Vz_elc[np.where(elcd_box_avg==np.max(elcd_box_avg))]),np.squeeze(Vy_elc[np.where(elcd_box_avg==np.max(elcd_box_avg))]),s = 40, marker = 'x', alpha = 1)
    axs[3].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[3].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[3].set_title(r'$<F_e(v_z,v_y)>_{z,y},$' + rf't = {time}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[3].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos3, ax=axs[3])
    cbar.ax.tick_params(labelsize=22)

    pos4 = axs[4].pcolormesh(Vz_ion/cSound0, Vy_ion/cSound0, iond_box_avg)
    axs[4].set_xlabel(r'$v_z/c_s$', fontsize=30)
    axs[4].set_ylabel(r'$v_y/c_s$', fontsize=30, labelpad=-1)
    axs[4].set_title(r'$<F_i(v_z,v_y)>_{z,y},$' + rf't = {time}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
    axs[4].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos4, ax=axs[4])
    cbar.ax.tick_params(labelsize=22)

    pos5 = axs[5].pcolormesh(grid_y, grid_z, n_ion)
    axs[5].set_xlabel(r'$y \ [d_e]$', fontsize=30)
    axs[5].set_ylabel(r'$z \ [d_e]$', fontsize=30, labelpad=-1)
    axs[5].set_title(r'$n_i(z,y)$', fontsize=30)
    axs[5].tick_params(labelsize = 26)
    cbar = fig.colorbar(pos5, ax=axs[5])
    cbar.ax.tick_params(labelsize=22)

    fig.tight_layout()
    plt.savefig(outDir+fileName+rf'_distributions_{fignum}.png', bbox_inches='tight')
    plt.close()

    #computing the fourier transform of the electric field
    if(fourier_transform):
#            e_z_raw_cell_average_k = np.zeros((nz_avg, ny_avg),dtype=complex)
#            J_z_k_raw              = np.zeros((nz_avg, ny_avg),dtype=complex)
        eField_fluct_z_k = np.fft.fftn(eField_fluct_z/vTe0)
        eField_fluct_y_k = np.fft.fftn(eField_fluct_y/vTe0)

        #print('fft_freq', np.fft.fftfreq(128) )
        J_fluct_z_k              = np.fft.fftn(J_fluct_z)
        J_fluct_y_k              = np.fft.fftn(J_fluct_y)

        if nFr-iFrame == 0:
            # eField_fluct_z_k_trace = np.zeros(eField_fluct_z_k.shape[0],eField_fluct_z_k.shape[1],nFrames )
            # eField_fluct_y_k_trace = np.zeros(eField_fluct_y_k.shape[0],eField_fluct_z_k.shape[1],nFrames )
            eField_fluct_z_k_trace = np.zeros((eField_fluct_z_k.shape[0],eField_fluct_z_k.shape[1], nFrames))
            eField_fluct_y_k_trace = np.zeros((eField_fluct_y_k.shape[0],eField_fluct_z_k.shape[1], nFrames))
        eField_fluct_z_k_trace[:, :, nFr-iFrame] = eField_fluct_z_k
        eField_fluct_y_k_trace[:, :, nFr-iFrame] = eField_fluct_y_k

        if times[nFr-iFrame] - times[iFrame-iFrame] > 2*3.141592*5*(1.1):
            t_idx_n = nFr-iFrame
            t_idx_back = np.argmin( np.absolute(times - (times[iFrame-iFrame] - 2*3.141592* m_ion**0.5 )) ) #get time index for the time 2 pi /omega_pi earlier
            eField_fluct_z_k_omega = np.fft.fftn(eField_fluct_z_k_trace[:, :, t_back:t_n], axes = 2)





        #e_z_raw_cell_average_k_y_int and J_z_k_raw_y_int are integrated over y
#            e_z_raw_cell_average_k_y_int = np.zeros(nz_avg)
#            J_z_k_raw_y_int                 = np.zeros(nz_avg)

#            e_z_raw_cell_average_k[0,0] = 0 #test: try to remove box-averaged component
#            e_y_raw_cell_average_k[0,0] = 0
#            J_z_k_raw[0,0] = 0
#            J_y_k_raw[0,0] = 0
        JdotE_k = np.abs(np.transpose(np.fft.fftshift(J_fluct_z_k*eField_fluct_z_k + J_fluct_y_k*eField_fluct_y_k) ) )
        eField_fluct_square_K = np.abs(np.transpose(np.fft.fftshift(eField_fluct_z_k**2 + eField_fluct_y_k**2) ) )
        #integrating over y direction (RLW: can we do this in a vectorized way?)
        #for j in range(ny_avg):
        #    for i in range(nz_avg):
#            for j in range(ny):
#                for i in range(nz):
#                    e_z_raw_cell_average_k_y_int[i] += np.abs(e_z_raw_cell_average_k[i,j])
#                    J_z_k_raw_y_int[i]              += np.abs(J_z_k_raw[i,j])


        #ek_t[nFr,:] = ek
        fignum = str(nFr).zfill(4)

        #z_plot_2d_sp   = np.linspace(-int(nz_avg/2), int(nz_avg/2-1), nz_avg)/lz
        #y_plot_2d_sp   = np.linspace(-int(ny_avg/2), int(ny_avg/2-1), ny_avg)/ly

        #         kz_plot_2d_sp   = 2.0*3.14159*vTe0*np.linspace(-int(nz/2), int(nz/2-1), nz)/lz
        #         ky_plot_2d_sp   = 2.0*3.14159*vTe0*np.linspace(-int(ny/2), int(ny/2-1), ny)/ly
        #         K_z, K_y = np.meshgrid(kz_plot_2d_sp, ky_plot_2d_sp, indexing = 'xy')


        #Plotting FFT of electric field and J

        fig, axs = plt.subplots(2,2,figsize=(20, 20), facecolor='w', edgecolor='k')
        fig.subplots_adjust(hspace = .5, wspace =.1)
        axs = axs.ravel()
        pos0 = axs[0].pcolormesh(Vz_elc, Vy_elc, elcd_box_avg)
        axs[0].scatter(boxavg_uElc_z, boxavg_uElc_y, s = 60)
        axs[0].scatter(np.squeeze(Vz_elc[np.where(elcd_box_avg==np.max(elcd_box_avg))]),np.squeeze(Vy_elc[np.where(elcd_box_avg==np.max(elcd_box_avg))]),s = 40, marker = 'x', alpha = 1)
        axs[0].set_xlabel(r'$v_z/c$', fontsize=30)
        axs[0].set_ylabel(r'$v_y/c$', fontsize=30, labelpad=-1)
        axs[0].set_title(r'$<F_e(v_z,v_y)>_{z,y},$' + rf't = {time}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
        axs[0].tick_params(labelsize = 26)
        cbar = fig.colorbar(pos0, ax=axs[0])
        cbar.ax.tick_params(labelsize=22)

        pos1 = axs[1].pcolormesh(Vz_ion, Vy_ion, iond_box_avg)
        axs[1].set_xlabel(r'$v_z/c$', fontsize=30)
        axs[1].set_ylabel(r'$v_y/c$', fontsize=30, labelpad=-1)
        axs[1].set_title(r'$<F_e(v_z,v_y)>_{z,y},$' + rf't = {time}'+ r' [$\omega_{pe}^{-1}$]', fontsize=26)
        axs[1].tick_params(labelsize = 26)
        cbar = fig.colorbar(pos1, ax=axs[1])
        cbar.ax.tick_params(labelsize=22)
#            pos0 = axs[0].plot(e_z_raw_cell_average_k_y_int)
#            axs[0].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
#            axs[0].set_ylabel(r'$E(k_z)$', fontsize=30, labelpad=-1)
#            axs[0].set_title(rf't = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[0].tick_params(labelsize = 26)
#            axs[0].set_yscale('symlog') #plot on log scale and take absolute value of number

#            pos1 = axs[1].plot(J_z_k_raw_y_int)
#            axs[1].set_xlabel(r'$k_z d_e$', fontsize=30)
#            axs[1].set_ylabel(r'$J(k_z)$', fontsize=30, labelpad=-1)
#            axs[1].set_title(rf't = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[1].tick_params(labelsize = 26)
#            axs[1].set_yscale('symlog')

        pos2 = axs[2].contourf(K_z, K_y, eField_fluct_square_K)
        axs[2].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
        axs[2].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
        axs[2].set_title(rf'$|\delta E^2|_k$, t = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
        axs[2].tick_params(labelsize = 26)
        cbar = fig.colorbar(pos2, ax=axs[2])
        cbar.ax.tick_params(labelsize=22)

        pos2 = axs[3].contourf(K_z, K_y, JdotE_k)
        axs[3].set_xlabel(r'$k_z \lambda_{De0}$', fontsize=30)
        axs[3].set_ylabel(r'$k_y \lambda_{De0}$', fontsize=30, labelpad=-1)
        axs[3].set_title(rf'$(\delta J \cdot \delta E)_k$, t = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
        axs[3].tick_params(labelsize = 26)
        cbar = fig.colorbar(pos3, ax=axs[3])
        cbar.ax.tick_params(labelsize=22)
#            pos3 = axs[3].contourf(K_z, K_y, JdotE_k )
#            axs[3].set_xlabel(r'$k_z d_e$', fontsize=30)
#            axs[3].set_ylabel(r'$k_y d_e$', fontsize=30, labelpad=-1)
#            axs[3].set_title(rf'$(\delta J \cdot \delta E)_k$, t = {time}'+ r'[$\omega_{pe}^{-1}$]', fontsize=30)
#            axs[3].tick_params(labelsize = 26)
#            cbar = fig.colorbar(pos3, ax=axs[3])
#            cbar.ax.tick_params(labelsize=22)

        fig.tight_layout()
        plt.savefig(outDir+fileName+rf'_fft_{fignum}.png', bbox_inches='tight')
        plt.close()

        #PLOTTING (to be done at each time step)

#### we have now left the loop over frames

Navg = 3
Rei = eField_boxavg_z + alpha*vTe0
#    E_over_J_rolling = np.zeros(nFrames)
for n in range(Navg,nFrames):
    E_over_J_rolling[n] = np.sum(eField_boxavg_z[n-Navg:n])/np.sum(J_boxavg_z[n-Navg:n])
for n in range(Navg):
    E_over_J_rolling[n] =  E_over_J_rolling[Navg]  #bfill the first Navg values


for n in range(Navg,nFrames):
    R_over_J_rolling[n] = np.sum(Rei[n-Navg:n])/np.sum(J_boxavg_z[n-Navg:n])
for n in range(Navg):
    R_over_J_rolling[n] =  R_over_J_rolling[Navg]  #bfill the first Navg values
nu_eff_rolling = np.absolute(R_over_J_rolling)

nu_Sagdeev = 0.025*np.absolute(J_boxavg_z/np.sqrt(Telc_boxavg))*Telc_boxavg/Tion_boxavg


#Energy plots
fig, axs = plt.subplots(2,3,figsize=(30, 20), facecolor='w', edgecolor='k')
fig.subplots_adjust(hspace = .5, wspace =.1)
axs = axs.ravel()
np.save(outDir+fileName+rf'_timesFile',times[0:fFrame-iFrame+1])
np.save(outDir+fileName+rf'_eZboxAvgFile',eField_boxavg_z[0:fFrame-iFrame+1])
np.save(outDir+fileName+rf'_jZboxAvgFile',J_boxavg_z[0:fFrame-iFrame+1])
np.save(outDir+fileName+rf'_eSquaredFile',eField_fluct_squared[0:fFrame-iFrame+1])
np.save(outDir+fileName+rf'_Tion_boxavgFile',Tion_boxavg[0:fFrame-iFrame+1])
np.save(outDir+fileName+rf'_Telc_boxavgFile',Telc_boxavg[0:fFrame-iFrame+1])


axs[0].plot(times[0:fFrame-iFrame+1],eField_fluct_squared,label=r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ T_{e0} $')
axs[0].plot(times[0:fFrame-iFrame+1],eField_fluct_squared_byT,label=r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ T_{e} $')
axs[0].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[0].set_ylabel(r'$ \left(\epsilon_0 \langle|\delta {E}|^2\rangle_{x,y}/2\right)/ T_{e0} $', fontsize=30)
axs[0].tick_params(labelsize = 26)
axs[0].legend(fontsize = 18)

axs[1].plot(times[0:fFrame-iFrame+1],eField_boxavg_z)
axs[1].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[1].set_ylabel(r'$\langle E_z \rangle$', fontsize=30)
axs[1].tick_params(labelsize = 26)
#    axs[1].set_ylim(top = 0.00)

axs[2].plot(times[0:fFrame-iFrame+1],E_over_J_rolling)
axs[2].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[2].set_ylabel(r'$\langle E_z\rangle /\langle\, J_z\rangle \ [\nu_{\mathrm{eff}}/ \omega_{pe}]$', fontsize=30)
axs[2].tick_params(labelsize = 26)
#    axs[2].set_ylim(0.0, 2*np.amax(E_over_J_rolling))

axs[3].plot(times[0:fFrame-iFrame+1],Telc_boxavg)
axs[3].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[3].set_ylabel(r'$T_e /T_{e0}$', fontsize=30)
axs[3].tick_params(labelsize = 26)
axs[3].set_ylim(0.0, 2*np.amax(Telc_boxavg) )

axs[4].plot(times[0:fFrame-iFrame+1],Tion_boxavg, label = r'$T_i /T_{e0}$')
axs[4].plot(times[0:fFrame-iFrame+1],Tion_boxavg/Telc_boxavg, label = r'$T_i /T_{e}$')
axs[4].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[4].set_ylabel(r'$T_i /T_{e0}$', fontsize=30)
axs[4].tick_params(labelsize = 26)
axs[4].set_ylim(0.0, 0.1)
axs[4].legend(fontsize = 18)
#    axs[4].set_ylim(0.0, 2*np.amax(Tion_boxavg) )

axs[5].plot(times[0:fFrame-iFrame+1],nu_eff_rolling, label = 'nu_eff')
axs[5].plot(times[0:fFrame-iFrame+1],nu_Sagdeev/10.0, label = 'nu_Sagdeev / 10')
axs[5].set_xlabel(r'$t \ [\omega_{pe}^{-1}]$', fontsize=30)
axs[5].set_ylabel(r'$\langle R_z\rangle /\langle\, J_z\rangle \ [\nu_{\mathrm{eff}}/ \omega_{pe}]$', fontsize=30)
axs[5].tick_params(labelsize = 26)
axs[5].legend(fontsize = 18)

fig.tight_layout()
plt.savefig(outDir+fileName+rf'_energy_current.png', bbox_inches='tight')
plt.close()

/Users/rlw/anaconda3/envs/pgkyl-dev/lib/python3.8/site-packages/numpy/core/_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)
<ipython-input-17-bb831677cd37>:4: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  time = hF['time'].read()
<ipython-input-29-78514b3b6a0a>:195: MatplotlibDeprecationWarning: shading='flat' when X and Y have the same dimensions as C is deprecated since 3.3.  Either specify the corners of the quadrilaterals with X and Y, or pass shading='auto', 'nearest' or 'gouraud', or set rcParams['pcolor.shading'].  This will become an error two minor releases later.
  pos0 = axs[0].pcolormesh(Vz_elc/cSound0, Vy_elc/cSound0, elcd_cut)
<ipython-input-29-7851

In [22]:
Tion_boxavg

array([0.0007285278565351, 0.0007285107363268, 0.000728316706575 ,
       0.0007274296069956, 0.0007250196415587, 0.0007195755307315,
       0.0007080135139546, 0.000687751636081 , 0.0006528874839582,
       0.0005902901455756])

In [23]:
Telc_boxavg

array([1.0000000822371364, 1.000007508750962 , 0.9998523053197653,
       0.9995953823476995, 0.9994752149011199, 0.9998156979832873,
       1.0012745011513844, 1.0047338580377547, 1.011735417843363 ,
       1.0244884058935981])

In [ ]:
Biskamp_newNoise_FixedJ_Tratio100_elc_0.bp

In [ ]:
def main_plotting(fileroot, max_output):
    print("Hello")
    

# Extra junk

In [9]:
#def main_plotting(fileroot, max_output):




if (auto_loading):

    for n in range(len(directories)):
        os.chdir(directories[n])

        input_files = []
        #loading up all input files
        for filename in os.listdir(os.getcwd()):
            if filename.endswith("~"):
                    os.remove(filename)
            elif '.lua' in filename:
                input_files.append(filename)
        #looping over all input files; selecting the fFrame and then plotting
        for i in range(len(input_files)):
            fileroot = input_files[i].split('.lua',1)[0]
            files_output_num = np.zeros(1)
            #looping over the output files to find the largest output time
            for filename in os.listdir(os.getcwd()):
                if (fileroot+'_elc') in filename:
                    output_number = re.findall(r'\d+.bp', filename)
                    if len(output_number)!=0:
                        output_number = int(output_number[0].split('.bp',1)[0])
                        files_output_num = np.append(files_output_num,output_number)
            max_output = int(np.max(files_output_num))
            if max_output==0:
                continue
            main_plotting(fileroot,max_output)
        os.chdir('..')
else:



Hello
